In [85]:
import os
import codecs
import pandas as pd
import chardet
import pandas as pd
import numpy as np
import jieba as jb
import re
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation
import gensim
from gensim import corpora
import pyLDAvis.gensim
import jieba.analyse
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble
import xgboost , textblob, string

In [86]:
app_reviews = pd.read_excel("app_reviews0429_whosecall.xlsx")

In [87]:
#定义删除除字母,数字，汉字以外的所有符号的函数
def remove_punctuation(line):
    line = str(line)
    if line.strip()=='':
        return ''
    rule = re.compile(u"[^\u4E00-\u9FA5]")
    line = rule.sub('',line)
    return line
 
#停用词列表
def stopwordslist(filepath):  
    stopwords = [line.strip() for line in open(filepath, 'r', encoding='utf-8').readlines()]  
    return stopwords  
 
#加载停用词
stopwords = stopwordslist("./chineseStopWords.txt")
jb.load_userdict('userDict.txt')

In [88]:
#删除除汉字以外的所有符号
app_reviews['clean_text'] = app_reviews['view'].apply(remove_punctuation)
 
#分词，并过滤停用词
app_reviews['cut_text'] = app_reviews['clean_text'].apply(lambda x: " ".join([w for w in list(jb.cut(x)) if w not in stopwords]))
app_reviews.head()

,user,date,title,view,rate_number,clean_text,cut_text
0,ArexChung,2018-10-03T08:04:56.000Z,未更新就關掉保護不合理,當資料有更新時，使用者未知情況下就自行關掉保護，待使用者更新完才又開啟，這造成了防護漏洞，已...,3,當資料有更新時使用者未知情況下就自行關掉保護待使用者更新完才又開啟這造成了防護漏洞已經有很多...,當資料 更新 時 使用者 未知情 況下 自行 關掉 保護 使用者 更新 完才 開啟 造成 防...
1,吹狗螺,2021-01-07T12:12:44.000Z,反饋,已經是進階會員了，每個月也自動扣款，常常自動封鎖號碼選項會無緣無故被取消，變成三不五時都要點...,5,已經是進階會員了每個月也自動扣款常常自動封鎖號碼選項會無緣無故被取消變成三不五時都要點進去看...,已經 進階會員 每個 月 自動 扣款 自動 封鎖 號碼 選項 無緣 無故 取消 變成 三不五...
2,雨謎,2020-05-06T03:22:06.000Z,應用程式無法打開,這個軟體，在通話上真的幫助了很多，可以過濾掉雜七雜八的電話，使我可以直接掛掉電話，但是自從上...,5,這個軟體在通話上真的幫助了很多可以過濾掉雜七雜八的電話使我可以直接掛掉電話但是自從上次更新之...,軟體 通話 真的 幫助 過濾 掉 雜七雜八 電話 使 直接 掛掉 電話 上次 更新 後 無法...
3,蔡小皮,2021-01-27T04:22:10.000Z,是很好用但也很困擾,能反查跟查詢店家號碼真的都蠻好用的，可是常常會出現需要啟用的情形，反覆開機刪除軟體真的很困擾...,5,能反查跟查詢店家號碼真的都蠻好用的可是常常會出現需要啟用的情形反覆開機刪除軟體真的很困擾希望...,反查 查詢 店家 號碼 真的 好用 出現 需要 啟用 情形 反覆 開機 刪除 軟體 真的 困...
4,Yuㄦ,2020-01-13T01:38:20.000Z,高風險顯示,很方便沒錯,5,很方便沒錯,方便 沒錯


In [89]:
%load_ext rpy2.ipython
#代表可以寫R

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [258]:
%%R -i app_reviews 
#%%R 整段code都變成R
#將app_reviews這個東西傳進來
library(stringr)
library(dplyr) 
app_reviews_category = app_reviews %>%
 mutate(efficiency = if_else(str_detect(view, '限制|效率|快速|慢|方便|快速|直接|便利|省時|好用|速度|快|無法使用'), true = '1', false = '0'))%>%
 mutate(satisfaction = if_else(str_detect(view, '開心|滿意|愉快|高興|困擾|錯誤|讚|用心|麻煩|不愉快|真好|好喜歡|爛|討厭|好厲害|好|不錯|差|擾人|白癡|惱人'), true = '1', false = '0'))%>%
 mutate(learnability = if_else(str_detect(view, '上手|好學|難用|指示|找不到|好找|找'), true = '1', false = '0'))%>%
 mutate(UI = if_else(str_detect(view, '介面|設計|美|醜|好看|漂亮|難看|插圖|美觀|版型|顯示'), true = '1', false = '0'))%>%
 mutate(functions = if_else(str_detect(view, '服務|使用|功能|登入|進階版|新增|增加|希望|建議|閃退|用|更新|下載|安裝|付費|權限|設定|花錢|訂閱|買'), true = '1', false = '0'))



In [259]:
%R -o app_reviews_category
app_reviews_category['binary'] = app_reviews_category['efficiency'] + app_reviews_category['satisfaction'] + app_reviews_category['learnability']+ app_reviews_category['UI']+ app_reviews_category['functions']

In [245]:
app_reviews_category

,user,date,title,view,rate_number,clean_text,cut_text,efficiency,satisfaction,learnability,UI,functions,binary
1,ArexChung,2018-10-03T08:04:56.000Z,未更新就關掉保護不合理,當資料有更新時，使用者未知情況下就自行關掉保護，待使用者更新完才又開啟，這造成了防護漏洞，已...,3,當資料有更新時使用者未知情況下就自行關掉保護待使用者更新完才又開啟這造成了防護漏洞已經有很多...,當資料 更新 時 使用者 未知情 況下 自行 關掉 保護 使用者 更新 完才 開啟 造成 防...,0,0,0,0,1,00001
2,吹狗螺,2021-01-07T12:12:44.000Z,反饋,已經是進階會員了，每個月也自動扣款，常常自動封鎖號碼選項會無緣無故被取消，變成三不五時都要點...,5,已經是進階會員了每個月也自動扣款常常自動封鎖號碼選項會無緣無故被取消變成三不五時都要點進去看...,已經 進階會員 每個 月 自動 扣款 自動 封鎖 號碼 選項 無緣 無故 取消 變成 三不五...,1,0,0,1,0,10010
3,雨謎,2020-05-06T03:22:06.000Z,應用程式無法打開,這個軟體，在通話上真的幫助了很多，可以過濾掉雜七雜八的電話，使我可以直接掛掉電話，但是自從上...,5,這個軟體在通話上真的幫助了很多可以過濾掉雜七雜八的電話使我可以直接掛掉電話但是自從上次更新之...,軟體 通話 真的 幫助 過濾 掉 雜七雜八 電話 使 直接 掛掉 電話 上次 更新 後 無法...,1,1,0,0,1,11001
4,蔡小皮,2021-01-27T04:22:10.000Z,是很好用但也很困擾,能反查跟查詢店家號碼真的都蠻好用的，可是常常會出現需要啟用的情形，反覆開機刪除軟體真的很困擾...,5,能反查跟查詢店家號碼真的都蠻好用的可是常常會出現需要啟用的情形反覆開機刪除軟體真的很困擾希望...,反查 查詢 店家 號碼 真的 好用 出現 需要 啟用 情形 反覆 開機 刪除 軟體 真的 困...,1,1,0,0,1,11001
5,Yuㄦ,2020-01-13T01:38:20.000Z,高風險顯示,很方便沒錯,5,很方便沒錯,方便 沒錯,1,0,0,0,0,10000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5396,還可以！希望有密碼鎖,2014-12-17T17:13:52.000Z,Ya,I phone終於有了,5,終於有了,終於有,0,0,0,0,0,00000
5397,Stranger ur papa,2014-12-18T01:18:34.000Z,唉,可惜又是閹割版，事後查詢雖然也不錯，但已經失去即時的意義，為何不考慮先在cydia上架呢...,3,可惜又是閹割版事後查詢雖然也不錯但已經失去即時的意義為何不考慮先在上架呢,可惜 閹割版 事後查詢 不錯 已經 失去 即時 意義 不 考慮 先 上架,0,1,0,0,0,01000
5398,LIN CHENG-MAO,2014-12-18T03:04:54.000Z,終於,等超久的ios,5,等超久的,超久,0,0,0,0,0,00000
5399,Yuwan.c,2014-12-18T04:36:52.000Z,好用！,"好厲害的APP,不下載太可惜！",5,好厲害的不下載太可惜,好 厲害 載太 可惜,0,1,0,0,1,01001


In [260]:
# 统计每个类别 样本数量
from collections import Counter
c = Counter(app_reviews_category['binary'])
Counter({k: c for k, c in c.items() if c < 10})

Counter({'01111': 5,
         '11110': 1,
         '10111': 5,
         '11111': 4,
         '01100': 2,
         '00110': 1,
         '10110': 1,
         '10100': 2})

In [261]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(analyzer='char',max_features=12000,min_df=10,ngram_range=(1,3))
# 获取训练样本的tfidf 向量化表示
#analyzer='word' --可能會取前後兩個字
X = tfidf.fit_transform(app_reviews_category['cut_text']) 
# n_samples : 表示文本行数，训练样本
# max_features : 词典长度 ＝ len(tfidf.get_feature_names())
print('n_samples = {},max_features = {}'.format(X.shape[0],X.shape[1]))


n_samples = 5400,max_features = 3278


In [262]:
print('feature size = ',len(tfidf.get_feature_names()))

feature size =  3278


In [263]:
category_count = np.unique(app_reviews_category['binary'])
print('class count = ',category_count)

class count =  ['00000' '00001' '00010' '00011' '00100' '00101' '00110' '00111' '01000'
 '01001' '01010' '01011' '01100' '01101' '01111' '10000' '10001' '10010'
 '10011' '10100' '10101' '10110' '10111' '11000' '11001' '11010' '11011'
 '11101' '11110' '11111']


In [264]:
# 统计tfidf 后，词汇列表
vocab_dict = tfidf.vocabulary_
print('词表大小:', len(vocab_dict)) # 表示字典的大小为720

vocab_dict = sorted(vocab_dict.items(),key = lambda d:d[1],reverse=True)
vocab_dict

词表大小: 3278


[('點進 ', 3277),
 ('點進', 3276),
 ('點 麻', 3275),
 ('點 不', 3274),
 ('點 ', 3273),
 ('點', 3272),
 ('黑', 3271),
 ('麼還', 3270),
 ('麼 ', 3269),
 ('麼', 3268),
 ('麻煩 ', 3267),
 ('麻煩', 3266),
 ('麻', 3265),
 ('鬼', 3264),
 ('高風險', 3263),
 ('高風', 3262),
 ('高', 3261),
 ('體 ', 3260),
 ('體', 3259),
 ('驟 ', 3258),
 ('驟', 3257),
 ('驗', 3256),
 ('騷擾 ', 3255),
 ('騷擾', 3254),
 ('騷', 3253),
 ('騙 電', 3252),
 ('騙 集', 3251),
 ('騙 ', 3250),
 ('騙', 3249),
 ('馬 ', 3248),
 ('馬', 3247),
 ('風險 ', 3246),
 ('風險', 3245),
 ('風', 3244),
 ('顯示 ', 3243),
 ('顯示', 3242),
 ('顯 示', 3241),
 ('顯 ', 3240),
 ('顯', 3239),
 ('類 ', 3238),
 ('類', 3237),
 ('題 請', 3236),
 ('題 無', 3235),
 ('題 更', 3234),
 ('題 已', 3233),
 ('題 回', 3232),
 ('題 不', 3231),
 ('題 ', 3230),
 ('題', 3229),
 ('顆 星', 3228),
 ('顆 ', 3227),
 ('顆', 3226),
 ('頭 ', 3225),
 ('頭', 3224),
 ('預期 ', 3223),
 ('預期', 3222),
 ('預', 3221),
 ('順利 ', 3220),
 ('順利', 3219),
 ('順', 3218),
 ('項 ', 3217),
 ('項', 3216),
 ('頁面 ', 3215),
 ('頁面', 3214),
 ('頁 ', 3213),
 ('頁', 3212),
 ('響 ', 321

In [265]:
# 样本中feature 通过tfidf 特征表示
X = tfidf.fit_transform(app_reviews_category['cut_text']) 
encoder = preprocessing.LabelEncoder()
y = encoder.fit_transform(app_reviews_category['binary'])

In [236]:
mapping = dict(zip(encoder.classes_, range(len(encoder.classes_))))
mapping


TypeError: 'Counter' object is not callable

In [266]:
#將數據集分為訓練集和驗證集
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.1, random_state=42)
print('X_train = ',X_train.toarray()[:2])
print('X_test = ',X_test.toarray()[:2])
print('y_train = ',y_train[:2])
print('y_test = ',y_test[:2])


X_train =  [[0.2013379  0.         0.         ... 0.         0.         0.        ]
 [0.21151538 0.         0.         ... 0.16226684 0.         0.        ]]
X_test =  [[0.1055154  0.         0.         ... 0.         0.         0.        ]
 [0.06628565 0.         0.         ... 0.         0.         0.        ]]
y_train =  [0 9]
y_test =  [1 0]


In [267]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB() 
model = clf.fit(X_train,y_train)
# 测试集预测
y_pred = model.predict(X_test)

# 绘制混淆矩阵
matrix = confusion_matrix(y_pred=y_pred,y_true=y_test)
#y_true 真實的label，一維數組格式，列名
#y_pred 模型預測的label，一維數組，行名
#labels 默認不指定，此時y_true、y_pred取並集，升序，做label
#sample_weight 樣本權重
#print(matrix)
# 获取的类别名称
category = list(mapping.keys())
# 矩阵中对角线是预测正确的记录数字，其他预测成别的类别了
pd.DataFrame(matrix,columns=category,index=category)

ValueError: Shape of passed values is (26, 26), indices imply (30, 30)

In [268]:
#模型準確率
accuracy = clf.score(X_test, y_test)
print('accuracy = ',accuracy)

accuracy =  0.5370370370370371


In [269]:
# 加载模型
import pickle
tf_model = '../tfidf_model.pkl'
with open(tf_model,'rb') as f:
    model_dict = pickle.load(f)
    model = model_dict['nb'] # 通过字典获取模型
    categories_labal_dict = model_dict['categories_label'] # 获取类别名称-编码关系
    tfidf_vec = model_dict['tfidfVectorizer'] # 获取文本特征向量转换器
    print('categories_labal_dict = ',categories_labal_dict)

categories_labal_dict =  {'00000': 0, '00001': 1, '00010': 2, '00011': 3, '00100': 4, '00101': 5, '00110': 6, '00111': 7, '01000': 8, '01001': 9, '01010': 10, '01011': 11, '01100': 12, '01101': 13, '01111': 14, '10000': 15, '10001': 16, '10010': 17, '10011': 18, '10100': 19, '10101': 20, '10110': 21, '10111': 22, '11000': 23, '11001': 24, '11010': 25, '11011': 26, '11101': 27, '11110': 28, '11111': 29}


In [68]:
#讀入模型
import pickle
with open('../tfidf_model.pkl','wb') as f:
    model_dict = {
        'categories_label' : mapping,
        'tfidfVectorizer' : tfidf,
        'nb' : model
    }
    pickle.dump(model_dict,f)


In [270]:
# 我们这里categories_labal_dict 进行kv 交换
labal_category_dict = {}
for k,v in categories_labal_dict.items():
    labal_category_dict[v]=k
print('labal_category_dict=',labal_category_dict)

# 封装文本特征-tfidf 数值表示的方法
def get_features(x):
    return tfidf_vec.transform(x)

labal_category_dict= {0: '00000', 1: '00001', 2: '00010', 3: '00011', 4: '00100', 5: '00101', 6: '00110', 7: '00111', 8: '01000', 9: '01001', 10: '01010', 11: '01011', 12: '01100', 13: '01101', 14: '01111', 15: '10000', 16: '10001', 17: '10010', 18: '10011', 19: '10100', 20: '10101', 21: '10110', 22: '10111', 23: '11000', 24: '11001', 25: '11010', 26: '11011', 27: '11101', 28: '11110', 29: '11111'}


In [271]:
import jieba
import warnings
warnings.filterwarnings('ignore')
text = "輸入生日的時候要用滾輪真的很不方便，然後常常在尖峰重要地點沒有車，上次趕時間的時候叫了服務，結果30分鐘才來，還有每次高鐵捷運台鐵三鐵共構車站總是沒有位置停，學校附近的車站常常沒車不然就是不能停，這樣服務真的真的有待加強，而且為什麼服務這樣4/1日起還要收錢？既然收錢的話服務不是應該從現在開始改善呀！"
words = [word for word in jieba.lcut(text) if len(word)>=2 and word not in stopwords]
print('words = ',words)
data = " ".join(words)
feat = get_features([data])# 使用加载模型后的tfidf 文本提取器
target = model.predict(feat)[0] # 使用加载后的model
print('target = ',target)
print('category_name = ',labal_category_dict[target])


words =  ['輸入', '生日', '滾輪', '真的', '不方便', '尖峰', '重要', '地點', '上次', '趕時間', '服務', '30', '分鐘', '每次', '高鐵捷', '運台', '鐵三鐵共構', '車站', '總是', '位置', '學校', '附近', '車站', '沒車', '不能', '服務', '真的', '真的', '有待', '加強', '服務', '日起', '還要', '收錢', '收錢', '服務', '不是', '應該', '現在', '改善']
target =  0
category_name =  00000
